# Implementaion of Linear Regression using Pytorch

### Problem Statement:
Create a model that predicts crop yields for apples and oranges (target variables) by looking at the average temperature, rainfall, and humidity (input variables or features) in a region. 

*   ***Approach 1*** : will implement the linear regression using self defined linear regression model, loss function (least mean square) and optimizer (gradient descent).
*   ***Approach 2***: will implement the linear regression by complete use of pytorch library.



# Approach 1

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Training Data
input_data =  np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

# Output Apple and Orange crop yield
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [ ]:
# converting the numpy data into torch tensor
input_data = torch.from_numpy(input_data)
targets = torch.from_numpy(targets)
print(input_data)
print(targets)

## Few important points regarding linear regression


*   In linear regression each target variable is estimated to be weighted sum of the input features/variables, offset by constant term called bias.
*   **Learning Part** of linear regression is to find out the weight and bias term using the training data so that accurate predications can be made for new data point.

*   Adjustment of weights will be done using the optimization technique called **gradient descent**. To apply gradient descent, first will calculate the loss function that determines how well our model is performing.

* **In case of mean squared error loss, loss is a quadratic function of our weights and biases, and our objective is to find the set of weights where the loss is the lowest.***

In [ ]:
# creating the weight and bias matrix for model
# intially we will pick the random value for weight and bias terms
weight = torch.randn((2,3), requires_grad=True)
bias = torch.randn(2, requires_grad=True)
print(weight)
print(bias)

In [ ]:
# formula for  linear regression model is
# y = W.transpose * X  + B
def linear_regression_model(inputs):
  return inputs @ weight.t() + bias


In [ ]:
# Defining the mean squared loss function

def mean_squared_loss(preds, target):
  diff = preds - target
  sq_diff_sum = torch.sum(diff*diff)
  avg_loss = sq_diff_sum/diff.numel()

  return avg_loss

In [ ]:

# Main algorithm
# This is an iterative algorithm and we will updates till the point where we able to achieve the minimum loss.
no_of_epochs = 160
learning_rate = 1e-5

for epoch in range(no_of_epochs):
  pred = linear_regression_model(input_data)
  loss = mean_squared_loss(pred, targets)
  loss.backward()

  with torch.no_grad():
    weight -= weight.grad * learning_rate
    bias -= bias.grad * learning_rate
    weight.grad.zero_()
    bias.grad.zero_()
  if (epoch+1) % 10 == 0:
    print('Epoch[{}/{}], Loss:{:.4f}'.format(epoch+1, no_of_epochs, loss))


# Approach 2

In [ ]:
import torch.nn as nn
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader


In [ ]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

In [ ]:
#converting the data into the tensor
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [ ]:
# now we will use the TENSORDATASET from pytorch utilis, which help us to access the rows from inputs and targets as tuples. 
# First element of tuple provide input variables for the selected rows and the second element gives the targets lables/values

train_ds = TensorDataset(inputs, targets)

In [ ]:
# DATALOADER is used to split the training data that we got from Tensordataset into the predefined batch size. 
batch_size = 5
train_batch = DataLoader(train_ds, batch_size, shuffle = True)

In [ ]:
# Define the model
# we can define the linear model using nn.linear function of pytorch. It will automatically create the weight and bias matrix. 
# Inputs given: no of Input features, no of target outputs

model = nn.Linear(3,2)
print(model.weight)
print(model.bias)

In [ ]:
# Define the loss function 
import torch.nn.functional as F

loss_function = F.mse_loss # mean squared loss

In [ ]:
# Optimizer
# Inputs : model parameters (i.e weight and bias matrix and learning rate)
learning_rate = 1e-5
opt = torch.optim.SGD(model.parameters(), learning_rate)


In [ ]:
# define training_function

def train_model (num_epoch, model, loss_fn, opt, train_batch):

  for epoch in range(num_epoch):
    
    # Taking the input and target data from defined batch
    for input, target in train_batch:
      
      # Generating Model predication
      pred = model(input)

      # Calculating loss
      loss = loss_fn(pred, target)

      # Computing gradients
      loss.backward()

      # updating the model parameters (weight and bias) using optimizer
      opt.step()

      # equating the model parameters gradients equal to zero for next epoch
      opt.zero_grad()

    if (epoch+1) % 10 == 0:
      # print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epoch, loss.item()))
      print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epoch, loss))


In [ ]:
train_model(100, model, loss_function, opt, train_batch)